In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf

#%pip install keras-tcn --no-dependencies

#%pip install tensorflow-addons[tensorflow-gpu] --no-dependencies

#%pip install pandas --upgrade

#%pip install multivariate_cwru

#%pip install tqdm

#%pip install scipy

#%pip install sklearn

#%pip install matplotlib

#%pip install typeguard


tf.config.list_physical_devices(device_type=None)
physical_devices = tf.config.list_physical_devices('GPU')
visible_devices = tf.config.get_visible_devices()
print(visible_devices)
num_GPU = len(tf.config.experimental.list_physical_devices('/physical_device:GPU:0'))
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics

#Deep Learning pkgs
from tensorflow.keras import backend as K, Input, Model, optimizers
from tensorflow.keras.layers import Dense, Activation
import tensorflow.keras.metrics
from tensorflow.keras.metrics import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint, Callback
from tensorflow.keras.activations import swish
K.backend()

# Python
from IPython.core.debugger import set_trace
from pathlib import Path
import os
import datetime
import time
import glob
from timeit import default_timer as timer


#Project Specific
import tcn_ed
from tcn_ed import TCN, tcn_full_summary, compiled_tcn
from help_pre import create_data_batcht as Create_Batch, create_pred_batch
import multivariate_cwru

def csv_import(dir_p, use_type, high_freq = False, b_size=1024):
    segment_length = 2400
    target = 'label'
    cwd = os.getcwd()
    col_ran = [*range(2400)]
    d_t = dict()
    d_t['label'] = 'int8'
    d_t['Unnamed: 0'] = 'int8'
    for col in col_ran:
        d_t['%s' % col] =  'float16'
    
    if str(cwd) == str(dir_p):
        dir_path=Path('./')
    else: dir_path = dir_p
    
    if use_type == 'train':
        if high_freq == False:
            train_frame = pd.read_csv((r'{}.csv').format('train_df_no48'), dtype=(d_t))
            test_frame = pd.read_csv((r'{}.csv').format('test_df_no48'), dtype=(d_t))
            
        if high_freq:
            train_frame = pd.read_csv((r'{}.csv').format('train_df_w48'), dtype=(d_t))
            test_frame = pd.read_csv((r'{}.csv').format('test_df_w48'), dtype=(d_t))
            
        train_frame = train_frame.set_index(train_frame.iloc[:, 0]).drop('Unnamed: 0', axis=1)
        train_target = train_frame.pop('label')
        train_x = train_frame.to_numpy().reshape(len(train_frame), segment_length, 1)
        train_y = train_target.to_numpy().reshape(-1, 1)
        options = tf.data.Options()
        options.experimental_optimization.apply_default_optimizations = True
        train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(b_size).with_options(options).cache()
        test_frame = test_frame.set_index(test_frame.iloc[:, 0]).drop('Unnamed: 0', axis=1)
        test_target = test_frame.pop('label')
        test_x = test_frame.to_numpy().reshape(len(test_frame), segment_length, 1)
        test_y = test_target.to_numpy().reshape(-1, 1)
        test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(b_size).with_options(options).cache()
        print('class balance of train frame: %s' % train_target.value_counts())
        print('class balance of validation (test) frame: %s' % test_target.value_counts())
        out_1 = train_dataset
        out_2 = test_dataset
            
    if use_type == 'pred':
        if high_freq == False:
            val_frame = pd.read_csv((r'{}.csv').format('pred_df_no48'), dtype=d_t)
            
        if high_freq:
            val_frame == pd.read_csv((r'{}.csv').format('pred_df_w48'), dtype=d_t)


        val_frame = val_frame.set_index(val_frame.iloc[:, 0]).drop('Unnamed: 0', axis=1)

        val_target = val_frame.pop('label')
        print('class balance of train frame: %s' % val_target.value_counts())
        val_x = val_frame.to_numpy().reshape(len(val_frame), segment_length, 1)
        val_y = val_target.to_numpy().reshape(-1, 1)

        options = tf.data.Options()
        options.experimental_optimization.apply_default_optimizations = True
        dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(b_size).with_options(options).cache()

        out_1 = dataset
        out_2 = val_y
        
    return out_1, out_2

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
train_dataset, test_dataset = csv_import(dir_p = os.getcwd(), use_type='train', high_freq = True)
val_dataset, predict_labels = csv_import(dir_p = os.getcwd(), use_type = 'pred' , high_freq = False)

class balance of train frame: 3    57451
1    28917
2    28747
0     9656
Name: label, dtype: int64
class balance of validation (test) frame: 3    30229
1    12081
2    11013
0     9650
Name: label, dtype: int64
class balance of train frame: 3    4770
1    2384
2    2384
0    1612
Name: label, dtype: int64


In [4]:
model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=4,
                         nb_filters=16,
                         kernel_size=11,
                         dilations=[2 ** i for i in range(7)],
                         nb_stacks=1,
                         max_len=2400,
                         output_len=1,  # type: int
                         padding='causal',  # type: str
                         use_skip_connections=True,  # type: bool
                         regression=False,  # type: bool
                         dropout_rate=0.3,  # type: float
                         name='Model1',  # type: str,
                         kernel_initializer='he_normal',  # type: str,
                         activation='swish',  # type:str,
                         opt='adam',
                         lr=0.002,
                         use_batch_norm=False,
                         use_layer_norm=False,
                         use_weight_norm=True)

[2021-05-12 05:45:18.279 tensorflow-2-3-gpu--ml-g4dn-xlarge-48a04204b31d97ed11a2a179d435:489 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-05-12 05:45:18.334 tensorflow-2-3-gpu--ml-g4dn-xlarge-48a04204b31d97ed11a2a179d435:489 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
x.shape= (None, 16)
model.x = (None, 2400, 1)
model.y = (None, 4)


In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2400, 1)]         0         
_________________________________________________________________
Model1 (TCN)                 (None, 16)                74286     
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 74,354
Trainable params: 37,332
Non-trainable params: 37,022
_________________________________________________________________


In [6]:
tcn_full_summary(model)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2400, 1)]         0         
_________________________________________________________________
matching_conv1D (Conv1D)     (None, 2400, 16)          32        
_________________________________________________________________
activation_3 (Activation)    (None, 2400, 16)          0         
_________________________________________________________________
weight_normalization (Weight (None, 2400, 16)          401       
_________________________________________________________________
activation (Activation)      (None, 2400, 16)          0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 2400, 16)          0         
_________________________________________________________________
weight_normalization_1 (Weig (None, 2400, 16)         

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.5,   
                              patience=3, 
                              min_lr=0.000000015,
                              verbose=1, 
                              cooldown=5)

ES_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                         min_delta=0.00001, 
                                         patience=70, 
                                         verbose=0, 
                                         mode='auto', 
                                         baseline=None, 
                                         restore_best_weights=True)



callback = [reduce_lr]#, ES_cb]


# Call .fit to train

```python
Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose=1,
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
```


In [ ]:
runs = 75

history = model.fit(train_dataset,
              epochs=runs,
              verbose=1,
              callbacks=callback,
              validation_data=test_dataset)

Epoch 1/75
122/122 [==============================] - 168s 1s/step - loss: 1.1260 - accuracy: 0.4642 - val_loss: 1.0087 - val_accuracy: 0.5142
Epoch 2/75
122/122 [==============================] - 164s 1s/step - loss: 0.9041 - accuracy: 0.5647 - val_loss: 0.7586 - val_accuracy: 0.6473
Epoch 3/75
122/122 [==============================] - 163s 1s/step - loss: 0.7488 - accuracy: 0.6438 - val_loss: 0.7287 - val_accuracy: 0.6536
Epoch 4/75
122/122 [==============================] - 163s 1s/step - loss: 0.6587 - accuracy: 0.6945 - val_loss: 0.6925 - val_accuracy: 0.6662
Epoch 5/75
122/122 [==============================] - 163s 1s/step - loss: 0.5863 - accuracy: 0.7352 - val_loss: 0.5951 - val_accuracy: 0.7396
Epoch 6/75
122/122 [==============================] - 163s 1s/step - loss: 0.5228 - accuracy: 0.7673 - val_loss: 0.5740 - val_accuracy: 0.7526
Epoch 7/75
122/122 [==============================] - 163s 1s/step - loss: 0.4784 - accuracy: 0.7891 - val_loss: 0.6093 - val_accuracy: 0.7554

In [14]:
Path

pathlib.Path

In [15]:
DATA_PATH

PosixPath('Data')

In [16]:
DE_path

PosixPath('Data/12k_DE')

In [17]:
df = matfile_to_df(DE_path)

In [18]:
df

,filename,DE_time,label
0,Data/12k_DE/OR007@3_0.mat,"[-0.16933862275449102, -0.04060878243512974, 0...",OR
1,Data/12k_DE/IR007_0.mat,"[-0.08300435129740519, -0.19573433133732535, 0...",IR
2,Data/12k_DE/IR021_0.mat,"[1.1894312375249503, -0.17786646706586826, -0....",IR
3,Data/12k_DE/B021_1.mat,"[0.23000814371257486, 0.18858718562874252, -0....",B
4,Data/12k_DE/IR014_2.mat,"[-0.02663936127744511, 0.02647692614770459, 0....",IR
...,...,...,...
59,Data/12k_DE/OR007@12_1.mat,"[-0.10623257485029941, 0.05246654690618762, 0....",OR
60,Data/12k_DE/B014_1.mat,"[0.005522794411177645, -0.10379604790419161, -...",B
61,Data/12k_DE/OR021@12_3.mat,"[0.04507574850299401, 0.11776546906187624, 0.0...",OR
62,Data/12k_DE/Normal_3.mat,"[0.014603076923076923, 0.05444861538461539, 0....",N


In [19]:
n_sample_points = []
for i in range(df.shape[0]):
        n_sample_points.append(len(df.iloc[i,1]))
        #print(n_sample_points)
n_sample_points = np.transpose(n_sample_points)

In [20]:
n_sample_points

array([122281, 121265, 122136, 121701, 121846, 121410, 121351, 121846,
       121351, 121556, 121846, 122136, 122136, 120984, 122136, 121991,
       121846, 483903, 122716, 121991, 122136, 122281, 485063, 121991,
       122426, 121556, 121846, 121991, 121991, 122917, 120801, 122281,
       243938, 120984, 121410, 121351, 122426, 122281, 122571, 121556,
       121846, 121991, 121991, 121701, 121991, 120801, 122136, 121991,
       122281, 121846, 122281, 121846, 121535, 122426, 122571, 122136,
       121701, 122136, 121846, 121991, 122136, 121701, 485643, 121556])

In [21]:
segment_length = 2400 # The length of the window used to take sub-arrays from each file
step_length = 300 # the amount the window moves along the sub-array

In [22]:
nmin1_sample_points = n_sample_points - segment_length
nmin1_remainder = nmin1_sample_points % step_length
nmin1_sample_points = nmin1_sample_points - nmin1_remainder
nminrem_sample_points = n_sample_points - nmin1_remainder
n_splits = nmin1_sample_points / step_length

splits = []
split_range = []
for i in n_splits:
    val = int(i+1)
    s_range = range(0, val, 1)
    split_range.append(s_range)
    
num_seq = []
num_seq += range(0, len(n_splits), 1)



In [23]:
nmin1_sample_points

array([119700, 118800, 119700, 119100, 119400, 118800, 118800, 119400,
       118800, 119100, 119400, 119700, 119700, 118500, 119700, 119400,
       119400, 481500, 120300, 119400, 119700, 119700, 482400, 119400,
       120000, 119100, 119400, 119400, 119400, 120300, 118200, 119700,
       241500, 118500, 118800, 118800, 120000, 119700, 120000, 119100,
       119400, 119400, 119400, 119100, 119400, 118200, 119700, 119400,
       119700, 119400, 119700, 119400, 119100, 120000, 120000, 119700,
       119100, 119700, 119400, 119400, 119700, 119100, 483000, 119100])

In [24]:
nmin1_remainder

array([181,  65,  36, 201,  46, 210, 151,  46, 151,  56,  46,  36,  36,
        84,  36, 191,  46,   3,  16, 191,  36, 181, 263, 191,  26,  56,
        46, 191, 191, 217, 201, 181,  38,  84, 210, 151,  26, 181, 171,
        56,  46, 191, 191, 201, 191, 201,  36, 191, 181,  46, 181,  46,
        35,  26, 171,  36, 201,  36,  46, 191,  36, 201, 243,  56])

In [25]:
nminrem_sample_points

array([122100, 121200, 122100, 121500, 121800, 121200, 121200, 121800,
       121200, 121500, 121800, 122100, 122100, 120900, 122100, 121800,
       121800, 483900, 122700, 121800, 122100, 122100, 484800, 121800,
       122400, 121500, 121800, 121800, 121800, 122700, 120600, 122100,
       243900, 120900, 121200, 121200, 122400, 122100, 122400, 121500,
       121800, 121800, 121800, 121500, 121800, 120600, 122100, 121800,
       122100, 121800, 122100, 121800, 121500, 122400, 122400, 122100,
       121500, 122100, 121800, 121800, 122100, 121500, 485400, 121500])

In [26]:
print(n_splits.shape, len(n_splits))
n_splits

(64,) 64


array([ 399.,  396.,  399.,  397.,  398.,  396.,  396.,  398.,  396.,
        397.,  398.,  399.,  399.,  395.,  399.,  398.,  398., 1605.,
        401.,  398.,  399.,  399., 1608.,  398.,  400.,  397.,  398.,
        398.,  398.,  401.,  394.,  399.,  805.,  395.,  396.,  396.,
        400.,  399.,  400.,  397.,  398.,  398.,  398.,  397.,  398.,
        394.,  399.,  398.,  399.,  398.,  399.,  398.,  397.,  400.,
        400.,  399.,  397.,  399.,  398.,  398.,  399.,  397., 1610.,
        397.])

In [27]:
num_seq = []
num_seq += range(0, len(n_splits), 1)

In [28]:
split_range[0]

range(0, 400)

In [29]:
for i in num_seq[0:2]:
    split_range_list = split_range[i]
    for segment in list(split_range_list):
        win_start = 0 + (step_length * segment)
        win_end = segment_length + (step_length * segment)
        #print(win_start, ':', win_end)
    #print('\n')

df_output

map_label = {'N':0, 'B':1, 'IR':2, 'OR':3}
df_output['label'] = df_output['label'].map(map_label)

df_output

find a way to add in overlapping samples

Number of windows should be able to be found by ((n_input_samples - window_size) - ((n_input_samples - window_size) % step) / step)

segment_length = 2400 # The length of the window used to take sub-arrays from each file
step_length = 300 # the amount the window moves along the sub-array

# Finished Function for Overlapping Window Split of Sequences

```python
def divide_signal(df, segment_length, step_length):
    n_sample_points = []
    splits = []
    split_range = []
    num_seq = []
    for i in range(df.shape[0]):
        n_sample_points.append(len(df.iloc[i,1]))
    n_sample_points = cp.transpose(n_sample_points)
    
    nmin1_sample_points = n_sample_points - segment_length
    nmin1_remainder = nmin1_sample_points % step_length
    nmin1_sample_points = nmin1_sample_points - nmin1_remainder
    nminrem_sample_points = n_sample_points - nmin1_remainder
    n_splits = nmin1_sample_points / step_length


    for i in n_splits:
        val = int(i+1)
        s_range = range(0, val, 1)
        split_range.append(s_range)
    
    
    num_seq += range(0, len(n_splits), 1)
    
    
    dic = {}
    idx = 0
    for i in num_seq:
        split_range_list = split_range[i]
        for segment in list(split_range_list):
            win_start = 0 + (step_length * segment)
            win_end = segment_length + (step_length * segment)
            dic[idx] = {
                'signal': df.iloc[i,1][win_start:win_end], # [0 + (step_length * (segment - 1)):segment_length + (step_length * segment)], 
                'label': df.iloc[i,2],
            }
            idx += 1
        df_tmp = pd.DataFrame.from_dict(dic,orient='index')
        df_output = pd.concat(
            [df_tmp[['label']], 
             pd.DataFrame(cp.vstack(df_tmp["signal"].values))
            ], 
            axis=1 )
    return df_output
```

In [30]:
# Test
segment_length = 2400
step_size = 300
df_out = get_df_all(DE_path, segment_length=segment_length, step_size=step_size)

In [31]:
df_out[16590:16595]

,label,0,1,2,3,4,5,6,7,8,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
16590,3,-0.529539,-0.856439,0.570959,0.779283,-0.258678,-0.190861,0.419895,0.213602,-0.433702,...,-0.220100,1.240192,-0.866998,-0.529539,1.144355,-0.632279,-1.174812,1.370140,0.858876,-1.596331
16591,3,0.008122,-0.097867,0.002437,0.181115,0.345175,-0.063350,-0.107207,0.209947,0.151065,...,-0.210760,0.118984,0.057664,-0.168933,-0.220912,0.032487,-0.008122,-0.174618,-0.034517,0.150252
16592,3,0.349236,-0.311063,-0.086903,0.194110,0.038172,-0.184364,-0.131572,0.194110,0.051573,...,0.119390,0.239592,-0.080405,-0.347205,0.097055,0.257866,-0.124669,-0.309033,0.147004,0.260302
16593,3,-0.476747,1.465977,-0.126293,-0.763851,0.631467,-0.030863,-0.278170,0.303348,0.276546,...,-0.118172,0.328525,0.233094,-1.061920,-0.055228,1.698259,-0.260302,-1.949628,1.176030,1.625976
16594,3,-0.200607,0.169745,0.351672,-0.174618,-0.332586,0.203044,0.153907,-0.276546,-0.170963,...,-0.197765,0.042233,-0.023553,-0.200201,-0.080811,0.214820,0.180709,-0.179085,0.062131,0.351672


In [32]:
df

,filename,DE_time,label
0,Data/12k_DE/OR007@3_0.mat,"[-0.16933862275449102, -0.04060878243512974, 0...",OR
1,Data/12k_DE/IR007_0.mat,"[-0.08300435129740519, -0.19573433133732535, 0...",IR
2,Data/12k_DE/IR021_0.mat,"[1.1894312375249503, -0.17786646706586826, -0....",IR
3,Data/12k_DE/B021_1.mat,"[0.23000814371257486, 0.18858718562874252, -0....",B
4,Data/12k_DE/IR014_2.mat,"[-0.02663936127744511, 0.02647692614770459, 0....",IR
...,...,...,...
59,Data/12k_DE/OR007@12_1.mat,"[-0.10623257485029941, 0.05246654690618762, 0....",OR
60,Data/12k_DE/B014_1.mat,"[0.005522794411177645, -0.10379604790419161, -...",B
61,Data/12k_DE/OR021@12_3.mat,"[0.04507574850299401, 0.11776546906187624, 0.0...",OR
62,Data/12k_DE/Normal_3.mat,"[0.014603076923076923, 0.05444861538461539, 0....",N


In [33]:
features = df_out.columns[1:]
target = 'label'

split_perc = 0.2

X_train, y_train = df_splitter(df_out, split_perc, target)

In [34]:
X_train.shape

(23654, 2400)

In [35]:
len(X_train)

23654

In [36]:
y_train.shape

(23654, 1)

In [37]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
24651,0.001299,0.166821,-0.198008,-0.277277,0.148303,0.188262,-0.171531,-0.095674,0.168445,-0.021441,...,0.051167,0.057989,0.019167,0.029888,0.070984,0.021279,0.051330,0.088527,-0.127674,-0.175268
16874,0.435790,-0.914711,-1.127113,-0.625405,-0.466307,-0.909015,-0.958656,-0.224202,0.142822,0.229085,...,-0.361734,-0.297851,-0.839435,-0.747475,0.787351,2.303461,1.966548,0.024414,-0.489908,1.301266
25876,-0.249825,-0.283449,-0.138557,0.189074,0.165034,0.090801,0.220749,0.310089,0.208079,-0.155126,...,-0.115329,-0.148791,-0.247226,-0.012508,0.077806,-0.261521,-0.337053,-0.010071,0.035573,-0.206617
25686,-0.377662,0.045157,0.663060,0.325682,0.044995,0.205318,0.219125,-0.164222,-0.319347,-0.266394,...,0.158537,0.083817,0.288972,0.226922,-0.040284,-0.216851,0.046294,0.222211,0.336728,0.058314
15281,0.081777,0.010222,-0.080734,-0.149994,-0.161677,-0.113487,-0.047564,0.022322,0.086367,0.128716,...,-0.051528,-0.011474,0.038802,0.074893,0.093668,0.094503,0.100553,0.092625,0.067174,0.014603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16008,1.336260,-1.719559,-1.242673,1.802567,2.371413,-1.688635,-3.161613,1.482744,3.783356,-1.080726,...,-1.368812,1.789546,2.086583,-1.466468,-2.953280,0.179850,2.678216,1.465654,-1.267900,-2.355137
10862,-0.038385,-0.021696,-0.027954,-0.049859,-0.065714,-0.038594,0.007927,0.050694,0.053823,0.011265,...,0.030041,0.023991,0.035882,0.016063,-0.031710,-0.106811,-0.166475,-0.167727,-0.120580,-0.054240
16660,0.224973,0.091370,0.000406,0.089339,0.192486,0.133603,-0.185988,-0.052385,0.046294,-0.036548,...,0.143755,-0.344362,0.102740,0.365885,-0.129948,-0.218881,0.158374,0.236343,-0.183146,-0.201420
19616,-0.115735,0.500706,0.244059,-0.296038,0.071878,0.317967,0.016650,-0.381723,-0.058883,0.259490,...,-0.396748,0.177866,0.447915,-0.259896,-0.428423,0.209947,0.350048,-0.214008,-0.251774,0.148222


In [38]:
X_train.iloc[0,:]

0       0.001299
1       0.166821
2      -0.198008
3      -0.277277
4       0.148303
          ...   
2395    0.021279
2396    0.051330
2397    0.088527
2398   -0.127674
2399   -0.175268
Name: 24651, Length: 2400, dtype: float64

In [39]:
y_train.iloc[0]

label    1
Name: 24651, dtype: int64

In [40]:
iter_range = [*range(0, len(X_train), 1)]
X_t, y_t = [],[]
for i in iter_range:
    v = X_train.iloc[i,:].values
    labels = y_train.iloc[i]
    X_t.append(v)        
    y_t.append(stats.mode(labels)[0][0])
X_t = np.array(X_t).reshape(23654, 2400, 1)
y_t = np.asarray(y_t).reshape(-1, 1)

In [41]:
X_t.shape

(23654, 2400, 1)

In [42]:
check_x = X_t[0:1].shape[1]
check_x

2400

In [43]:
X_t

array([[[ 1.29948104e-03],
        [ 1.66820878e-01],
        [-1.98008423e-01],
        ...,
        [ 8.85271457e-02],
        [-1.27674012e-01],
        [-1.75267505e-01]],

       [[ 4.35789900e-01],
        [-9.14711200e-01],
        [-1.12711300e+00],
        ...,
        [ 2.44140000e-02],
        [-4.89907600e-01],
        [ 1.30126620e+00]],

       [[-2.49825230e-01],
        [-2.83449301e-01],
        [-1.38557166e-01],
        ...,
        [-1.00709780e-02],
        [ 3.55732934e-02],
        [-2.06617485e-01]],

       ...,

       [[ 2.24972655e-01],
        [ 9.13697605e-02],
        [ 4.06087824e-04],
        ...,
        [ 2.36343114e-01],
        [-1.83145609e-01],
        [-2.01419561e-01]],

       [[-1.15735030e-01],
        [ 5.00706287e-01],
        [ 2.44058782e-01],
        ...,
        [-2.14008283e-01],
        [-2.51774451e-01],
        [ 1.48222056e-01]],

       [[ 9.09563077e-02],
        [ 5.36141538e-02],
        [ 6.07070769e-02],
        ...,
        

In [44]:
y_t

array([[1],
       [2],
       [3],
       ...,
       [3],
       [3],
       [0]])

In [45]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_t)

y_train = enc.transform(y_t)

In [46]:
y_train.shape

(23654, 4)

# TCN
## Finished with data processing, now need to run through model. 

* Find a way to insert as batches. The batches should run in the following process.
    1. For the `batch_size` loop
        * take the first sequence `X_t[i]`
        * for the length of `X_t[i]`
            * 


# TCN code is shown below

In [47]:
model=Sequential()

from tcn import TCN, tcn_full_summary, compiled_tcn
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# if time_steps > tcn_layer.receptive_field, then we should not
# be able to solve this task.
batch_size, time_steps, input_dim = 64, segment_length, 1


# def get_x_y(size=1000):
    #import numpy as np
    #pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    #x_train = np.zeros(shape=(size, time_steps, 1))
    #y_train = np.zeros(shape=(size, 1))
    #x_train[pos_indices, 0] = 1.0  # we introduce the target in the first timestep of the sequence.
    #y_train[pos_indices, 0] = 1.0  # the task is to see if the TCN can go back in time to find it.
    #return x_train, y_train


tcn_layer = TCN(input_shape=(time_steps, input_dim))
# The receptive field tells you how far the model can see in terms of timesteps.
print('Receptive field size =', tcn_layer.receptive_field)

m = Sequential([
    tcn_layer,
    Dense(1)
])

m.compile(optimizer='adam', loss='mse')

tcn_full_summary(m)

# x, y = get_x_y()
# m.fit(x, y, epochs=10, validation_split=0.2)

from tcn import TCN, tcn_full_summary, compiled_tcn
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=4,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=X_t[0:1].shape[1],
                         output_len=1,  # type: int
                         padding='causal',
                         use_weight_norm=True,
                         use_skip_connections=True)